In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy import cellreader
from cellpy.utils import ocv_rlx
import holoviews as hv

%matplotlib inline
hv.extension('bokeh')

In [ ]:
######################################################################
##                                                                  ##
##                       development                                ##
##                                                                  ##
######################################################################

if os.name == 'nt':
    # Use these when working on my work PC:
    raw_data_path = r"C:\Scripting\MyFiles\development_cellpy\dev_data\gitt"
    out_data_path = r"C:\Scripting\MyFiles\development_cellpy\out"
else:
    # Use these when working on my MacBook:
    raw_data_path = "/Users/jepe/scripting/cellpy/dev_data/gitt"
    out_data_path = "/Users/jepe/scripting/cellpy/dev_data/out"

raw_data_path = Path(raw_data_path)
out_data_path = Path(out_data_path)

print(" SETTING SOME PRMS ".center(80, "="))
prms.Paths["db_filename"] = "cellpy_db.xlsx"
prms.Paths["cellpydatadir"] = out_data_path
prms.Paths["outdatadir"] = out_data_path
prms.Paths["rawdatadir"] = raw_data_path
prms.Paths["db_path"] = out_data_path
prms.Paths["filelogdir"] = out_data_path
pprint(prms.Paths)

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
fn = "20190403_cen59_04_rateGITT_01"
mass = 0.3
resn = fn + ".res"
cellpyn = fn + ".h5"
filename = prms.Paths["rawdatadir"] / resn
cellpyname = prms.Paths["cellpydatadir"] / cellpyn

In [ ]:
cell = cellreader.get(filename, mass=mass, logging_mode="INFO")
cell.save(cellpyname)
cell = cellreader.get(cellpyname)

In [ ]:
# print(cell)

## Some functions

### Custom step-table for GITT
Need to allow for several ocv steps with same step number in the same cycle


In [ ]:
# THIS IS NOW IMPLEMENTED IN CELLPY
def _ustep(n):  # NEW
    un = []
    c = 1
    n = n.diff()
    for i in n:
        if i != 0:
            c += 1
        un.append(c)
    return un

In [ ]:
# THIS IS NOW IMPLEMENTED IN CELLPY
# make_step_table(all_steps=True)

def make_step_table_custom(cell, do_masking=False):
## copy-pasted from cellpy.cellreader and updated

    print(" Create step table (with u-steps) ".center(80, "="))
    nhdr = cell.headers_normal
    shdr = cell.headers_step_table
    shdr.u_step = "ustep"  # NEW
    
    
    df = cell.dataset.raw

    def first(x):
        return x.iloc[0]

    def last(x):
        return x.iloc[-1]

    def delta(x):
        if x.iloc[0] == 0.0:
            # starts from a zero value
            difference = 100.0 * x.iloc[-1]
        else:
            difference = (x.iloc[-1] - x.iloc[0]) * 100 / abs(x.iloc[0])

        return difference

    keep = [
        nhdr.data_point_txt,
        nhdr.test_time_txt,
        nhdr.step_time_txt,
        nhdr.step_index_txt,
        nhdr.cycle_index_txt,
        nhdr.current_txt,
        nhdr.voltage_txt,
        nhdr.ref_voltage_txt,
        nhdr.charge_capacity_txt,
        nhdr.discharge_capacity_txt,
        nhdr.internal_resistance_txt,
        # "ir_pct_change"
    ]

    # only use col-names that exist:
    keep = [col for col in keep if col in df.columns]

    df = df[keep]
    df[nhdr.sub_step_index_txt] = 1
    rename_dict = {
        nhdr.cycle_index_txt: shdr.cycle,
        nhdr.step_index_txt: shdr.step,
        nhdr.sub_step_index_txt: shdr.sub_step,
        nhdr.data_point_txt: shdr.point,
        nhdr.test_time_txt: shdr.test_time,
        nhdr.step_time_txt: shdr.step_time,
        nhdr.current_txt: shdr.current,
        nhdr.voltage_txt: shdr.voltage,
        nhdr.charge_capacity_txt: shdr.charge,
        nhdr.discharge_capacity_txt: shdr.discharge,
        nhdr.internal_resistance_txt: shdr.internal_resistance,
    }

    df = df.rename(columns=rename_dict)
    df[shdr.u_step]=_ustep(df[shdr.step])  # NEW

    by = [shdr.cycle, shdr.step, shdr.sub_step, shdr.u_step]  # UPDATED

    gf = df.groupby(by=by)
    df_steps = (gf.agg(
        [np.mean, np.std, np.amin, np.amax, first, last, delta]
    ).rename(columns={'amin': 'min', 'amax': 'max', 'mean': 'avr'}))

    df_steps = df_steps.sort_values(by=[(shdr.point, "first")])  # NEW
    df_steps = df_steps.reset_index()

    df_steps[shdr.type] = np.nan
    df_steps[shdr.sub_type] = np.nan
    df_steps[shdr.info] = np.nan

    current_limit_value_hard = cell.raw_limits["current_hard"]
    current_limit_value_soft = cell.raw_limits["current_soft"]
    stable_current_limit_hard = cell.raw_limits["stable_current_hard"]
    stable_current_limit_soft = cell.raw_limits["stable_current_soft"]
    stable_voltage_limit_hard = cell.raw_limits["stable_voltage_hard"]
    stable_voltage_limit_soft = cell.raw_limits["stable_voltage_soft"]
    stable_charge_limit_hard = cell.raw_limits["stable_charge_hard"]
    stable_charge_limit_soft = cell.raw_limits["stable_charge_soft"]
    ir_change_limit = cell.raw_limits["ir_change"]

    mask_no_current_hard = (
        df_steps.loc[:, (shdr.current, "max")].abs()
        + df_steps.loc[:, (shdr.current, "min")].abs()
    ) < current_limit_value_hard

    mask_voltage_down = df_steps.loc[:, (shdr.voltage, "delta")] < \
        - stable_voltage_limit_hard

    mask_voltage_up = df_steps.loc[:, (shdr.voltage, "delta")] > \
        stable_voltage_limit_hard

    mask_voltage_stable = df_steps.loc[:, (shdr.voltage, "delta")].abs() < \
        stable_voltage_limit_hard

    mask_current_down = df_steps.loc[:, (shdr.current, "delta")] < \
        - stable_current_limit_soft

    mask_current_up = df_steps.loc[:, (shdr.current, "delta")] > \
        stable_current_limit_soft

    mask_current_negative = df_steps.loc[:, (shdr.current, "avr")] < \
        - current_limit_value_hard

    mask_current_positive = df_steps.loc[:, (shdr.current, "avr")] > \
        current_limit_value_hard

    mask_galvanostatic = df_steps.loc[:, (shdr.current, "delta")].abs() < \
        stable_current_limit_soft

    mask_charge_changed = df_steps.loc[:, (shdr.charge, "delta")].abs() > \
        stable_charge_limit_hard

    mask_discharge_changed = df_steps.loc[:, (shdr.discharge, "delta")].abs() > \
        stable_charge_limit_hard

    mask_no_change = (df_steps.loc[:, (shdr.voltage, "delta")] == 0) & \
        (df_steps.loc[:, (shdr.current, "delta")] == 0) & \
        (df_steps.loc[:, (shdr.charge, "delta")] == 0) & \
        (df_steps.loc[:, (shdr.charge, "delta")] == 0)

    if do_masking:  # NEW
        print("-masking and labelling steps")
        df_steps.loc[mask_no_current_hard & mask_voltage_stable,
                     shdr.type] = 'rest'

        df_steps.loc[mask_no_current_hard & mask_voltage_up,
                     shdr.type] = 'ocvrlx_up'

        df_steps.loc[mask_no_current_hard & mask_voltage_down,
                     shdr.type] = 'ocvrlx_down'

        df_steps.loc[mask_discharge_changed & mask_current_negative,
                     shdr.type] = 'discharge'

        df_steps.loc[mask_charge_changed & mask_current_positive,
                     shdr.type] = 'charge'

        df_steps.loc[
            mask_voltage_stable & mask_current_negative & mask_current_down,
            shdr.type
        ] = 'cv_discharge'

        df_steps.loc[mask_voltage_stable & mask_current_positive &
                     mask_current_down, shdr.type] = 'cv_charge'

        # --- internal resistance ----
        df_steps.loc[mask_no_change, shdr.type] = 'ir'
        # assumes that IR is stored in just one row

        # --- sub-step-txt -----------
        df_steps[shdr.sub_type] = None

    # check if all the steps got categorizes
    print("-looking for un-categorized steps")  # REMARK! uses print instead of logging
    empty_rows = df_steps.loc[df_steps[shdr.type].isnull()]
    if not empty_rows.empty:
        print(
            f"found {len(empty_rows)}",
            f":{len(df_steps)} non-categorized steps ",
            f"(please, check your raw-limits)",)

    print(f"-flatten columns")
    flat_cols = []
    for col in df_steps.columns:
        if isinstance(col, tuple):
            if col[-1]:
                col = "_".join(col)
            else:
                col = col[0]
        flat_cols.append(col)

    df_steps.columns = flat_cols

    return df_steps

#### Checking

In [ ]:
gt = make_step_table_custom(cell, do_masking=True)

In [ ]:
gt.columns

In [ ]:
gt[gt.cycle==5].head(10)

## Initial exploration

- Should plot all cycles.
- Should allow for "zoomin" into / selecting a single cycle.

### Overview of full cycle

In [ ]:
t, v = cell.get_timestamp(), cell.get_voltage()

In [ ]:
all_cycs = hv.Curve((t,v), ('t', 'time (sec)'), ('v', 'voltage (v)'), label="voltage-time").opts()

In [ ]:
steptable = cell.dataset.steps

In [ ]:
cycle_label_df = steptable.drop_duplicates("cycle")

In [ ]:
cycle_labels = hv.Labels((cycle_label_df.test_time_first, cycle_label_df.voltage_first, cycle_label_df.cycle))

In [ ]:
%%opts Curve [width=1200, xformatter="%6.0f"]
all_cycs * cycle_labels

### Full cycle in "holomap" form

In [ ]:
# creating a dictionary of curves (for each cycle)
cycs_dict = dict()
for c in cell.get_cycle_numbers():
    t = cell.get_timestamp(cycle=c)
    t = t - np.amin(t.values)  # setting first point to t=0
    curve = hv.Curve((t, cell.get_voltage(cycle=c)), ("time", "time (seconds)"), ("voltage", "voltage (v vs. Li/Li+)"))
    cycs_dict[c] = curve

In [ ]:
%%opts Curve [width=800, xformatter="%6.0f"]

hmap = hv.HoloMap(cycs_dict,"cycle")
hmap

In [ ]:
%%opts Curve [width=800, xformatter="%6.0f"]
(all_cycs * cycle_labels + hmap).cols(1)

## Processing cycle

In [ ]:
dfdata = cell.dataset.raw

In [ ]:
cyc5 = dfdata.loc[dfdata.Cycle_Index == 5, :]
gt5 = gt.loc[gt.cycle==5, :]

In [ ]:
gt5.head(40)

In [ ]:
cyc5curve = hv.Curve(cyc5,  ("Test_Time", "time"), ("Voltage", "voltage")).opts(color="grey", alpha=0.5)
cyc5points = hv.Scatter(cyc5,  ("Test_Time", "time"), ("Voltage", "voltage")).opts(size=5, fill_alpha=0.2)

In [ ]:
slabels5 = hv.Labels((gt5.test_time_first, gt5.voltage_first, gt5.ustep))

In [ ]:
%%opts Curve [width=1000, height=600]
cyc5curve * cyc5points * slabels5

In [ ]:
# estimate some values

# electrolyte-electrode area (cm2)
radius = 0.75  # cm
roughness = 1.0
print("Should probably use BET to find out this?")
area = (1*roughness) * np.pi * (radius**2)
print(f"S: {area:6.2f} cm2")

# number of moles
# 1 mol Si weighs 28.0855 g
mass = mass
number_of_moles = 28.0855 / mass 
print(f"n_m: {number_of_moles:6.2f} mol")

# molar volume
# The 2006 CODATA recommended value for the molar volume of silicon is 12.0588349(11)×10−6 m3/mol, with a relative standard uncertainty of 9.1×10−8
molar_volume = 12.06 # cm3/mol
print(f"V_m: {molar_volume:6.2f} cm3/mol")


In [ ]:
# approximating electrode-electrolyte contact area

diameter_si = 200.0 * 10e-7  # cm
rho_si = 2.32  # g/cm3
mass = 0.2 * 0.001  # g
A = (2*3/diameter_si) * (mass/rho_si)
print(f"Diameter: {diameter_si:8.6f} cm")
print(f"Mass:     {mass:8.6f} g")
print(f"Calculated contact area (ideal case): {A:8.6f} cm2")

# For reference (v, a, m is pr Si particle)
v = (4/3)*np.pi*((diameter_si/2)**3)
m = rho_si * v
n = mass / m
a = 4 * np.pi * (diameter_si/2)**2 


In [ ]:
def calc_A(n_m=93.62, V_m=12.06, S=35.0):
    """
    D = 4 /((pi)(tau)) * (n_m * V_m / S)^2 * (DEs / DEt)^2 = A /tau * (DEs / DEt)^2
    A = 4 /(pi) * (n_m * V_m / S)^2
    
    tau: duration of the current pulse (s)  
    n_m: number of moles (mol)  
    V_m: molar volume of electrode (cm3/mol)  
    S: electrode-electrolyte contact area (cm2)  
    DEs: steady state voltage change due to the current pulse (V)  
    DEt: voltage change during the constant current pulse (eliminating the iR drop) (V)  
    
    Ref.: application note from Metrohm Autolab b.v. pdf (BAT03)
    """
    A = (4 / np.pi) * (n_m * V_m / S)**2
    return A

In [ ]:
def auto_calc_D(steptable, cycle_number, A=1.0, tau=None, ustep_first=None, ustep_last=None):
    """Function for extracting diffusion constant(s) and inserting into steptable.
    
    D = 4 /((pi)(tau)) * (n_m * V_m / S)^2 * (DEs / DEt)^2 = A /tau * (DEs / DEt)^2
    A = 4 /(pi) * (n_m * V_m / S)^2
    
    """
    st = None
    if cycle_number is None:
        # This function is intended to only work on a pr. cycle basis
        # to prevent users from "poluting" the steptable for "non_GITT" experiments.
        print("no cycle number given")
        return
    
    st = steptable[steptable.cycle==cycle_number]
    if st.empty:
        print("the given cycle is not found")
        return
    
    if ustep_first is not None:
        st = st[st.ustep>=ustep_first]
    if ustep_last is not None:
        st = st[st.ustep<=ustep_last]
    
    # used for finding DE
    n3 = st["voltage_last"].shift(periods=-3)
    n1 = st["voltage_first"].shift(periods=-1)
    
    # used for finding tau
    if tau is None:
        tau = st["step_time_delta"]
    
    # used for validating if proper GITT step
    t0 = st["type"]
    t1 = st["type"].shift(periods=-1)
    t2 = st["type"].shift(periods=-2)
    t3 = st["type"].shift(periods=-3)
    
    st["valid_D"] = (t0==t2) & (t1.str.contains("ocv"))
    
    # calculating
    st["DEt"] = st["voltage_last"] - st["voltage_first"]
    st["DEs"] = n3 - n1
    st["D"] = A / tau * (st["DEs"] / st["DEt"])**2
    
    return st

In [ ]:
A = calc_A()
x = auto_calc_D(gt, 5, A=A, ustep_first=27)
discharge = x.loc[(x.type=="discharge") & (x.valid_D)]
charge = x.loc[(x.type=="charge") & (x.valid_D)]
# discharge.head(25)

In [ ]:
%%opts Curve [width=600 xformatter="%6.0f"]
X = discharge["discharge_avr"]*1_000_000/mass
Y = discharge["D"]
discharge_diffcurve = hv.Scatter((X,Y), ("capacity"), ("diffusion"), label="discharge").opts(size=12) * hv.Curve((X,Y)).opts(alpha=0.4)

X = charge["charge_avr"]*1_000_000/mass
Y = charge["D"]
charge_diffcurve = hv.Scatter((X,Y), ("capacity"), ("diffusion"), label="charge").opts(size=12) * hv.Curve((X,Y)).opts(alpha=0.4)

discharge_diffcurve * charge_diffcurve

## Picking out titration curves

In [ ]:
cycle = 5
titcurve_ustep = 29

# filtering wrt cycle number to get the Data_Points for the step
datapoints = discharge.loc[discharge.ustep==titcurve_ustep, ["point_first", "point_last"]]
datapoints

In [ ]:
# Using the data points to select the data from dfdata
first = datapoints.iloc[0, 0]
last = datapoints.iloc[0, 1]
dftit = dfdata.loc[first:last, :]
dftit.head()

In [ ]:
dfcurve = hv.Curve(dftit,  ("Step_Time", "time"), ("Voltage", "voltage")).opts(color="grey", alpha=0.5).opts(width=1000, xformatter="%6.0f")
min_time = hv.VLine(100.0)
max_time = hv.VLine(1000.0)
a = hv.Arrow(x=500, y = 0.31, text="fit", direction= "v")
dfcurve * min_time * max_time * a

In [ ]:
# plotting dE vs sqrt(t) and doing a linear regression to find the slope
dE = dftit["Voltage"]
t = np.sqrt(dftit["Step_Time"])

In [ ]:
decurve = hv.Curve((t, dE), "sqrt-time", "voltage")
decurve

# Temporary stuff (will probably be deleted)

### Methodology for finding diff coeffs
#### Equation
```
Eq. 1.1.
D = (4 / pi) (i*V_m / (Z_A*F*S)^2 * ((dE/dd)/(dE/d(sqrt(t)))^2  

i: current (A)  
Z_A: charge number  
F: Faraday´s constant (96_458 C/mol)
V_m: molar volume of electrode (cm3/mol)  
S: electrode-electrolyte contact area (cm2)  

(dE/dd): the slope of the coulometric titration curve, found by plotting the
steady state voltages E (V) measured after each titration step δ

steady state voltage change due to the current pulse (V)  
(dE/d(sqrt(t)):  the slope of the linearized plot of the potential E (V) during the current pulse of duration t (s).  
```



#### step 1
Need to find the slope of E vs δ. This is the same as finding dV/dQ. One way to do this is to fit the "relaxed" voltage curve (the OCV_RLX points) to a polynomial and diffing it.
#### step 2
Then we also need to find the slope of E vs time during the pulse. For this we can e.g. use the polytfit function from scipy. The first 100 seconds should not be used as they
are typically from charge-transfere resistance and the concentration overpotential.
#### step 3
Now it is time to get all the other prms (where the most difficult one is the electrode-electrolyte contact area). It should ideally be found using BET. But I guess the best is to calculate it using a sensible model, or from some scattering experiments (SANS?).

## Example
Based on application note from Metrohm Autolab b.v. pdf (BAT03) 


```
Eq. 1.1.
D = (4 / pi) (i*V_m / (Z_A*F*S)^2 * ((dE/dd)/(dE/d(sqrt(t)))^2  

i: current (A)  
Z_A: charge number  
F: Faraday´s constant (96_458 C/mol)
V_m: molar volume of electrode (cm3/mol)  
S: electrode-electrolyte contact area (cm2)  

(dE/dd): the slope of the coulometric titration curve, found by plotting the
steady state voltages E (V) measured after each titration step δ

steady state voltage change due to the current pulse (V)  
(dE/d(sqrt(t)):  the slope of the linearized plot of the potential E (V) during the current pulse of duration t (s).  
```


```
Eq. 1.2.
D = 4 /((pi)(tau)) * (n_m * V_m / S)^2 * (DEs / DEt)^2  

tau: duration of the current pulse (s)  
n_m: number of moles (mol)  
V_m: molar volume of electrode (cm3/mol)  
S: electrode-electrolyte contact area (cm2)  
DEs: steady state voltage change due to the current pulse (V)  
DEt: voltage change during the constant current pulse (eliminating the iR drop) (V)  
```

### Example code:

```python

D = (A / tau) * (DEs / DEt)**2
# during charge
DEt = ustep_52.voltage[-1] - ustep_52.voltage[0]  # charge step
DEs = ustep_55.voltage[-1] - ustep_53.voltage[0]  # ocv steps
```

```python
# during discharge
DEt = ustep_27.voltage[-1] - ustep_27.voltage[0]  # discharge step
DEs = ustep_30.voltage[-1] - ustep_28.voltage[0]  # ocv steps
```

Using step-table

```python
DEt = ustep[n].voltage_last - ustep[n].voltage_first  # charge step
DEs = ustep[n+3].voltage_last - ustep[n+1].voltage_first # ocv steps
```

In [ ]:
def simple_calc_of_d(steptable, c=5, n=27, A=1.0):
    # This is an implementation with no ambitions with regards to speed or usability
    gt = steptable[steptable.cycle==c]
    ustep_n = gt.loc[gt.ustep==n, :]
    ustep_n_1 = gt.loc[gt.ustep==n+1, :]
    ustep_n_3 = gt.loc[gt.ustep==n+3, :]

    v = ustep_n.voltage_first.values
    DEt_n = ustep_n.voltage_last.values - ustep_n.voltage_first.values
    DEs_n = ustep_n_3.voltage_last.values - ustep_n_1.voltage_first.values

    Dn = A * (DEs_n / DEt_n)**2

    return v[0], Dn[0]

In [ ]:
V = list()
D = list()
print("Voltage  Diffusion")
for n in range(27, 45, 2):
    v, d = simple_calc_of_d(gt, 5, n)
    V.append(v)
    D.append(d)
    print(f"({n}) {v:7.2}: {d:7.2}")

In [ ]:
diffcurve = hv.Scatter((V,D), ("voltage"), ("diffusion")).opts(size=12) * hv.Curve((V,D)).opts(alpha=0.4)
diffcurve